# RAPIDS & Scanpy Single-Cell RNA-seq Workflow

Copyright (c) 2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License") you may not use this file except in compliance with the License. You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0 

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This notebook demonstrates a single-cell RNA analysis workflow that begins with preprocessing a count matrix of size `(n_gene, n_cell)` and results in a visualization of the clustered cells for further analysis.

For demonstration purposes, we use a dataset of ~70,000 human lung cells from Travaglini et al. 2020 (https://www.biorxiv.org/content/10.1101/742320v2) and label cells using the ACE2 and TMPRSS2 genes. See the README for instructions to download this dataset.

## Import requirements

In [11]:
import numpy as np
import scanpy as sc
import anndata

import time

import cudf
import cupy as cp

from cuml.decomposition import PCA
from cuml.manifold import TSNE
from cuml.cluster import KMeans

import rapids_scanpy_funcs

import warnings
warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

In [12]:
## SCVi Imports
import scvi
from typing import Tuple
import matplotlib.pyplot as plt

We use the RAPIDS memory manager on the GPU to control how memory is allocated.

In [13]:
import rmm

rmm.reinitialize(
    managed_memory=True, # Allows oversubscription
    pool_allocator=False, # default is False
    devices=0, # GPU device IDs to register. By default registers only GPU 0.
)

cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

## Input data

In the cell below, we provide the path to the `.h5ad` file containing the count matrix to analyze. Please see the README for instructions on how to download the dataset we use here.

We recommend saving count matrices in the sparse .h5ad format as it is much faster to load than a dense CSV file. To run this notebook using your own dataset, please see the README for instructions to convert your own count matrix into this format. Then, replace the path in the cell below with the path to your generated `.h5ad` file.

In [14]:
input_file = "../data/krasnow_hlca_10x.sparse.h5ad"

## Set parameters

In [15]:
# marker genes
RIBO_GENE_PREFIX = "RPS" # Prefix for ribosomal genes to regress out
markers = ["ACE2", "TMPRSS2", "EPCAM"] # Marker genes for visualization

# filtering cells
min_genes_per_cell = 200 # Filter out cells with fewer genes than this expressed 
max_genes_per_cell = 6000 # Filter out cells with more genes than this expressed 

# filtering genes
n_top_genes = 5000 # Number of highly variable genes to retain

# PCA
n_components = 50 # Number of principal components to compute

# t-SNE
tsne_n_pcs = 20 # Number of principal components to use for t-SNE

# k-means
k = 35 # Number of clusters for k-means

# KNN
n_neighbors = 15 # Number of nearest neighbors for KNN graph
knn_n_pcs = 50 # Number of principal components to use for finding nearest neighbors

# UMAP
umap_min_dist = 0.3 
umap_spread = 1.0

# Gene ranking
ranking_n_top_genes = 50 # Number of differential genes to compute for each cluster

In [16]:
start = time.time()

## Load and Prepare Data

We load the sparse count matrix from an `h5ad` file using Scanpy. The sparse count matrix will then be placed on the GPU. 

In [17]:
data_load_start = time.time()

In [18]:
%%time
adata = sc.read(input_file)

CPU times: user 145 ms, sys: 315 ms, total: 460 ms
Wall time: 458 ms


We maintain the index of unique genes in our dataset:

In [19]:
%%time
genes = cudf.Series(adata.var_names)
sparse_gpu_array = cp.sparse.csr_matrix(adata.X)

CPU times: user 163 ms, sys: 264 ms, total: 427 ms
Wall time: 431 ms


Verify the shape of the resulting sparse matrix:

In [20]:
sparse_gpu_array.shape

(65662, 26485)

And the number of non-zero values in the matrix:

In [21]:
sparse_gpu_array.nnz

126510394

In [22]:
data_load_time = time.time()
print("Total data load and format time: %s" % (data_load_time-data_load_start))

Total data load and format time: 0.9365050792694092


## Preprocessing

In [23]:
preprocess_start = time.time()

### Filter

We filter the count matrix to remove cells with an extreme number of genes expressed.

In [24]:
%%time
sparse_gpu_array = rapids_scanpy_funcs.filter_cells(sparse_gpu_array, min_genes=min_genes_per_cell, max_genes=max_genes_per_cell)

CPU times: user 506 ms, sys: 312 ms, total: 819 ms
Wall time: 872 ms


Some genes will now have zero expression in all cells. We filter out such genes.

In [25]:
%%time
sparse_gpu_array, genes = rapids_scanpy_funcs.filter_genes(sparse_gpu_array, genes, min_cells=1)

CPU times: user 935 ms, sys: 108 ms, total: 1.04 s
Wall time: 1.06 s


The size of our count matrix is now reduced.

In [26]:
sparse_gpu_array.shape

(65462, 22058)

In [27]:
%%time
adata_org = anndata.AnnData(sparse_gpu_array.get())
adata_org.layers["counts"] = adata_org.X.copy()

CPU times: user 316 ms, sys: 171 ms, total: 486 ms
Wall time: 486 ms


In [28]:
print (sparse_gpu_array)

  (0, 17)	2.0
  (0, 24)	1.0
  (0, 47)	1.0
  (0, 52)	2.0
  (0, 63)	1.0
  (0, 67)	1.0
  (0, 70)	1.0
  (0, 96)	1.0
  (0, 102)	1.0
  (0, 114)	2.0
  (0, 115)	2.0
  (0, 158)	1.0
  (0, 199)	1.0
  (0, 215)	1.0
  (0, 257)	1.0
  (0, 276)	1.0
  (0, 298)	2.0
  (0, 318)	4.0
  (0, 320)	3.0
  (0, 329)	1.0
  (0, 343)	1.0
  (0, 353)	1.0
  (0, 386)	2.0
  (0, 388)	1.0
  (0, 404)	1.0
  :	:
  (65461, 21795)	1.0
  (65461, 21811)	2.0
  (65461, 21813)	2.0
  (65461, 21817)	8.0
  (65461, 21821)	1.0
  (65461, 21832)	1.0
  (65461, 21835)	1.0
  (65461, 21838)	1.0
  (65461, 21841)	1.0
  (65461, 21848)	1.0
  (65461, 21873)	1.0
  (65461, 21874)	2.0
  (65461, 21903)	5.0
  (65461, 21904)	1.0
  (65461, 21912)	4.0
  (65461, 21925)	1.0
  (65461, 21938)	1.0
  (65461, 21958)	1.0
  (65461, 21967)	6.0
  (65461, 21978)	1.0
  (65461, 21979)	1.0
  (65461, 21997)	15.0
  (65461, 22008)	1.0
  (65461, 22013)	1.0
  (65461, 22023)	1.0


### Normalize

We normalize the count matrix so that the total counts in each cell sum to 1e4.

In [29]:
%%time
sparse_gpu_array = rapids_scanpy_funcs.normalize_total(sparse_gpu_array, target_sum=1e4)

CPU times: user 0 ns, sys: 2.35 ms, total: 2.35 ms
Wall time: 2.81 ms


Next, we log transform the count matrix.

In [30]:
%%time
sparse_gpu_array = sparse_gpu_array.log1p()

CPU times: user 88.5 ms, sys: 72.4 ms, total: 161 ms
Wall time: 164 ms


### Select Most Variable Genes

We will now select the most variable genes in the dataset. However, we first save the 'raw' expression values of the ACE2 and TMPRSS2 genes to use for labeling cells afterward. We will also store the expression of an epithelial marker gene (EPCAM).

In [31]:
%%time
tmp_norm = sparse_gpu_array.tocsc()
marker_genes_raw = {
    ("%s_raw" % marker): tmp_norm[:, genes[genes == marker].index[0]].todense().ravel()
    for marker in markers
}

del tmp_norm

CPU times: user 625 ms, sys: 280 ms, total: 905 ms
Wall time: 1.02 s


Now, we convert the count matrix to an annData object.

In [32]:
%%time
adata = anndata.AnnData(sparse_gpu_array.get())
adata.var_names = genes.to_pandas()

CPU times: user 213 ms, sys: 72.3 ms, total: 285 ms
Wall time: 286 ms


Using scanpy, we filter the count matrix to retain only the 5000 most variable genes.

In [33]:
%%time
sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes, flavor="cell_ranger")
adata = adata[:, adata.var.highly_variable]

CPU times: user 1.09 s, sys: 7.88 ms, total: 1.1 s
Wall time: 1.12 s


### Regress out confounding factors (number of counts, ribosomal gene expression)

We can now perform regression on the count matrix to correct for confounding factors -  for example purposes, we use the number of counts and the expression of ribosomal genes. Many workflows use the expression of mitochondrial genes (named starting with `MT-`).

In [34]:
ribo_genes = adata.var_names.str.startswith(RIBO_GENE_PREFIX)

We now calculate the total counts and the percentage of ribosomal counts for each cell.

In [35]:
%%time
n_counts = adata.X.sum(axis=1)
percent_ribo = (adata.X[:,ribo_genes].sum(axis=1) / n_counts).ravel()

n_counts = cp.array(n_counts).ravel()
percent_ribo = cp.array(percent_ribo).ravel()

CPU times: user 944 ms, sys: 41.5 ms, total: 986 ms
Wall time: 984 ms


And perform regression:

In [36]:
%%time
sparse_gpu_array = cp.sparse.csc_matrix(adata.X)

CPU times: user 744 ms, sys: 140 ms, total: 884 ms
Wall time: 882 ms


In [37]:
%%time
sparse_gpu_array = rapids_scanpy_funcs.regress_out(sparse_gpu_array, n_counts, percent_ribo)

CPU times: user 46.1 s, sys: 8.23 s, total: 54.3 s
Wall time: 56.3 s


### Scale

Finally, we scale the count matrix to obtain a z-score and apply a cutoff value of 10 standard deviations, obtaining the preprocessed count matrix.

In [38]:
%%time
sparse_gpu_array = rapids_scanpy_funcs.scale(sparse_gpu_array, max_value=10)

CPU times: user 210 ms, sys: 142 ms, total: 352 ms
Wall time: 360 ms


In [39]:
preprocess_time = time.time()
print("Total Preprocessing time: %s" % (preprocess_time-preprocess_start))

Total Preprocessing time: 64.54650330543518


## SCVi training function

In [40]:
def compute_scvi_latent(
    adata: sc.AnnData,
    n_latent: int = 5,
    n_epochs: int = 100,
    lr: float = 1e-3,
    use_batches: bool = False,
    use_cuda: bool = False,
):
    """Train and return a scVI model and sample a latent space

    :param adata: sc.AnnData object non-normalized
    :param n_latent: dimension of the latent space
    :param n_epochs: number of training epochs
    :param lr: learning rate
    :param use_batches
    :param use_cuda
    :return: (scvi.Posterior, latent_space)
    """
    # Convert easily to scvi dataset
    scvi.data.setup_anndata(adata, layer="counts")
    # Train a model
    vae = scvi.model.SCVI(
        adata,
        n_latent=n_latent,
        use_cuda=use_cuda
    )
    vae.train(n_epochs=n_epochs, train_size=0.9)

    latent = vae.get_latent_representation()

    return latent

## Cluster & Visualize

We store the preprocessed count matrix as an AnnData object, which is currently in host memory. We also add the expression levels of the marker genes as observations to the annData object.

In [41]:
#%%time
genes = adata.var_names
adata_layers = adata.layers
adata = anndata.AnnData(sparse_gpu_array.get())
adata.var_names = genes
adata.layers = adata_layers

for name, data in marker_genes_raw.items():
    adata.obs[name] = data.get()
    
print (adata)

AnnData object with n_obs × n_vars = 65462 × 5000
    obs: 'ACE2_raw', 'TMPRSS2_raw', 'EPCAM_raw'


## SCVI LATENTS

In [42]:
%%time
n_epochs = 5

# use_cuda to use GPU
use_cuda = True

scvi_latent = compute_scvi_latent(
    adata_org, n_epochs=n_epochs, n_latent=10, use_cuda=use_cuda
)
adata.obsm["X_scvi"] = scvi_latent
    

INFO      Training is still in warming up phase. If your applications rely on the posterior  
          quality, consider training for more epochs or reducing the kl warmup.              
INFO      Training time:  622 s. / 5 epochs                                                  
CPU times: user 1h 2min 38s, sys: 49min 7s, total: 1h 51min 46s
Wall time: 10min 27s


In [ ]:
print (scvi_latent)

In [ ]:
elbo_train_set = trainer.history["elbo_train_set"]
elbo_test_set = trainer.history["elbo_test_set"]
print (trainer.history)
%matplotlib inline
print (len(elbo_train_set))
x = np.linspace(0, n_epochs, (len(elbo_train_set)))
plt.plot(x, elbo_train_set, label="train")
plt.plot(x, elbo_test_set, label="test")
plt.ylim(4000, 90000)
plt.legend()

### Reduce

We use PCA to reduce the dimensionality of the matrix to its top 50 principal components.

In [ ]:
#%%time
#adata.obsm["X_pca"] = PCA(n_components=n_components, output_type="numpy").fit_transform(adata.X)

### T-SNE + k-Means

We next use t-SNE on the top 20 principal components to visualize the cells in two dimensions.

In [ ]:
%%time
adata.obsm['X_tsne'] = TSNE().fit_transform(adata.obsm["X_scvi"][:,:10])

We cluster the cells using k-means on the principal components. For example purposes, we set k=35.

In [ ]:
%%time
kmeans = KMeans(n_clusters=k, random_state=0).fit(adata.obsm['X_scvi'])
adata.obs['kmeans'] = kmeans.labels_.astype(str)

We visualize the cells using t-SNE and label cells by color according to the k-means clustering.

In [ ]:
%%time
sc.pl.tsne(adata, color=["kmeans"])

Separately, we label the cells using the 'raw' expression of the EPCAM, ACE2 and TMPRSS2 marker genes.

In [ ]:
%%time
sc.pl.tsne(adata, size=4, color=["ACE2_raw"], color_map="Blues", vmax=1, vmin=-0.05)
sc.pl.tsne(adata, size=4, color=["TMPRSS2_raw"], color_map="Blues", vmax=1, vmin=-0.05)
sc.pl.tsne(adata, size=4, color=["EPCAM_raw"], color_map="Reds", vmax=1,  vmin=-0.05)

### UMAP + Graph clustering

We can also visualize the cells using the UMAP algorithm in Rapids. Before UMAP, we need to construct a k-nearest neighbors graph in which each cell is connected to its nearest neighbors. This can be done conveniently using rapids functionality already integrated into Scanpy.

Note that Scanpy uses an approximation to the nearest neighbors on the CPU while the GPU version performs an exact search. While both methods are known to yield useful results, some differences in the resulting visualization and clusters can be observed.

In [ ]:
%%time
sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=knn_n_pcs, method='rapids')

The UMAP function from Rapids is also integrated into Scanpy.

In [ ]:
%%time
sc.tl.umap(adata, min_dist=umap_min_dist, spread=umap_spread, method='rapids')

Next, we use the Louvain algorithm for graph-based clustering, once again using the `rapids` option in Scanpy.

In [ ]:
%%time
sc.tl.louvain(adata, flavor='rapids')

We plot the cells using the UMAP visualization, and using the Louvain clusters as labels.

In [ ]:
%%time
sc.pl.umap(adata, color=["louvain"])

We can also use the Leiden clustering method in RAPIDS. This method has not been integrated into Scanpy and needs to be called separately.

In [ ]:
%%time
adata.obs['leiden'] = rapids_scanpy_funcs.leiden(adata)

In [ ]:
%%time
sc.pl.umap(adata, color=["leiden"])

We can also view cells using UMAP and labeling by raw EPCAM, ACE2 and TMPRSS2 expression.

In [ ]:
%%time
sc.pl.umap(adata, size=4,color=["ACE2_raw"], color_map="Blues", vmax=1, vmin=-0.05)
sc.pl.umap(adata, size=4, color=["TMPRSS2_raw"], color_map="Blues", vmax=1, vmin=-0.05)
sc.pl.umap(adata, size=4, color=["EPCAM_raw"], color_map="Reds", vmax=1, vmin=-0.05)

## Differential expression analysis

Once we have done clustering, we can compute a ranking for the highly differential genes in each cluster. Here we use the Louvain clusters as labels.

In [ ]:
cluster_labels = cudf.Series.from_categorical(adata.obs["louvain"].cat)
genes = cudf.Series(genes)

We use logistic regression to identify the top 50 genes distinguishing each cluster.

In [ ]:
%%time
scores, names, reference = rapids_scanpy_funcs.rank_genes_groups(
    sparse_gpu_array, 
    cluster_labels, 
    genes, 
    n_genes=ranking_n_top_genes, groups='all', reference='rest')

In [ ]:
%%time
adata.uns["rank_genes_groups"] = {}
adata.uns["rank_genes_groups"]["params"] = dict(groupby="louvain", method="logreg", reference=reference, use_raw=False)
adata.uns["rank_genes_groups"]['scores'] = scores
adata.uns["rank_genes_groups"]['names'] = names
sc.pl.rank_genes_groups(adata, n_genes=20)

## Create Zoomed View

The speedup offered by Rapids makes it easy to interactively re-analyze subsets of cells. To illustrate this, we select epithelial cells (EPCAM+) from the dataset.

In [ ]:
reanalysis_start = time.time()

In [ ]:
%%time
print (adata)
epcam_cells = adata.obs["EPCAM_raw"] > 0.0
adata = adata[epcam_cells,:]
adata.shape
print (adata)

We can repeat the dimension reduction, clustering and visualization using this subset of cells in seconds.

In [ ]:
%%time
adata=adata.copy()
scvi_latent = compute_scvi_latent(
    adata, n_epochs=n_epochs, n_latent=10, use_cuda=use_cuda
)
adata.obsm["X_scvi"] = scvi_latent
#adata.obsm["X_pca"] = PCA(n_components=n_components, output_type="numpy").fit_transform(adata.X)
sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=knn_n_pcs, method='rapids')
sc.tl.umap(adata, min_dist=umap_min_dist, spread=umap_spread, method='rapids')
adata.obs['leiden'] = rapids_scanpy_funcs.leiden(adata)

And plot the results:

In [ ]:
%%time
sc.pl.umap(adata, color=["leiden"])
sc.pl.umap(adata, color=["ACE2_raw"], color_map="Blues", vmax=1, vmin=-0.05)
sc.pl.umap(adata, color=["TMPRSS2_raw"], color_map="Blues", vmax=1, vmin=-0.05)

In [ ]:
reanalysis_time = time.time()
print("Total reanalysis time : %s" % (reanalysis_time-reanalysis_start))

In [ ]:
print("Full time: %s" % (time.time() - start))